Write a python code in cocalc that crack the used random number generator, based on the following scenario:

Assume you got a ciphertext (in hexa) that  is  b'62716b6b777862677362667e7b',  as a result of XOR encryption.
Additionally, assume that you know the original encrypted plaintext was 'cybersecurity'
You know that the key were random numbers generated by LCG.
By knowing the above information, crack the LCG parameters.
 


# Imports

In [1]:
from functools import reduce
from binascii import unhexlify

# Setup

we use `xor`, `egcd`, `modinv` `gcd` from the previous lectures.

In [2]:
def xor(string1, string2):
    return "".join([chr(ord(c1) ^ ord(c2)) for (c1,c2) in zip(string1,string2)])

In [3]:
def egcd(a,b):
    if a ==0 :
        return (b,0,1)
    else:
        g,y,x = egcd(b%a,a)
        return (g,x-(b//a)*y,y)
    
def modinv(b,n):
    g,x,_ = egcd(b,n)
    if g == 1:
        return x%n

In [4]:
def gcd(x,y):
    while y:
        x,y = y, x%y
    return x

# LCG cracking algorithm
We use the algorithm from the lecture slides. The algorithm is as follows:

1. Compute the modulus of the LCG using the key values.
2. Compute the multiplier of the LCG using the key values and modulus.
3. Compute the increment of the LCG using the key values, modulus, and multiplier.

In [5]:
def crack_unknown_increment(states, modulus, multiplied):
    increment = (states[1]-states[0]*multiplied) % modulus
    return increment

In [6]:
def crack_unknown_multiplier(states, modulus):
    multiplier = (states[2]-states[1]) * modinv(states[1]-states[0],modulus) % modulus
    increment = crack_unknown_increment(states, modulus, multiplier)
    return increment, multiplier

In [7]:
def crack_unknown_modulus(states):
    diffs = [s1 - s0 for s0, s1 in zip(states, states[1:])]
    zeroes = [t2*t0 - t1*t1 for t0, t1, t2 in zip(diffs, diffs[1:], diffs[2:])]
    modulus = abs(reduce(gcd, zeroes))
    increment, multiplier = crack_unknown_multiplier(states, modulus)
    return modulus, increment, multiplier

# Solution
We get the cyphertext and plaintext from the problem statement. We unhexify the cyphertext. We then get the key by XORing the cyphertext and plaintext. Assuming that the key is generated by the LCG, we can use the key to crack the LCG parameters.

In [8]:
def getLCGValues(cypher_hex,message):
    unhexifiedCypherText = unhexlify(cypher_hex).decode()
    print("unhexified cypher text: ",unhexifiedCypherText)
    values = xor(unhexifiedCypherText, message)
    keys = [ord(c) for c in values]
    print(f"Keys used for encrypting {message}:",keys)
    return crack_unknown_modulus(keys)

### Getting the LCG values


In [9]:
cypher_hex = b'62716b6b777862677362667e7b'
message = "cybersecurity"
(modulus,increment, multiplier) = getLCGValues(cypher_hex,message)

print("modulus: ", modulus, "\nincrement: ", increment, "\nmultiplier: ", multiplier)

unhexified cypher text:  bqkkwxbgsbf~{
Keys used for encrypting cybersecurity: [1, 8, 9, 14, 5, 11, 7, 4, 6, 16, 15, 10, 2]
modulus:  17 
increment:  3 
multiplier:  5


## Testing
After we got our LCG modulus, increment, multiplier values, we can check if those values actually generate "random" sequence used in the encryption of the message

We use a bit modified LCG algorithm where the first state is the seed itself and we pass the LCG parameters as an arguments of a function.

In [10]:
def prng_lcg(seed, repeat, multiplier, increment, modulus):
    state = seed
    numbers = [seed]
    for _ in range(repeat-1):
        state = (state * multiplier+increment) % modulus
        numbers.append(state)
    return numbers

In [11]:
generatedKeyList = prng_lcg(1,len(message),multiplier,increment,modulus)
print (generatedKeyList)

[1, 8, 9, 14, 5, 11, 7, 4, 6, 16, 15, 10, 2]


Let's compare our generated Key list and key list we got after using xor for cyphertext and message.

In [12]:
unhexifiedCypherText = unhexlify(cypher_hex).decode()
values = xor(unhexifiedCypherText, message)
originalKeyList = [ord(c) for c in values]

In [13]:
def checkIfEqualKeyLists(originalKeyList, newKeyList):
    areEqual = True
    for (ind,_) in enumerate(originalKeyList):
        if originalKeyList[ind] != newKeyList[ind]:    
            areEqual = False
    if areEqual:
        print("The lists are equal")
    else:
        print("The lists are not equal")

In [14]:
checkIfEqualKeyLists(originalKeyList,generatedKeyList)

The lists are equal
